## Importing the CIFAR-100 dataset

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data(label_mode='fine')

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [2]:
import os

os.getcwd()

'/kaggle/working'

In [3]:
os.chdir('/kaggle/working/logs/fit')
print(os.getcwd())

/kaggle/working/logs/fit


In [4]:
os.listdir()

['20230426-164547',
 '20230423-151812',
 '20230423-144521',
 '20230427-131337',
 '20230426-163456',
 '20230427-132245',
 '20230426-163559',
 '20230526-094034',
 '20230525-165119',
 '20230503-094139',
 '20230529-180355',
 '20230429-165728',
 '20230426-162513',
 '20230427-130858',
 '20230427-135240',
 '20230529-085257',
 '20230427-134859',
 '20230427-130948',
 '20230526-113717',
 '20230503-083530',
 '20230529-121437',
 '20230530-072223',
 '20230525-215217',
 '20230429-165620',
 '20230427-135703',
 '20230503-173904',
 '20230423-142813']

In [ ]:
print(tf.config.list_physical_devices('GPU'))
!nvidia-smi

In [ ]:
from keras.applications.inception_v3 import preprocess_input
from tensorflow.data import Dataset
from keras.layers import Resizing, Rescaling
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

enc = LabelBinarizer()

y_train = enc.fit_transform(y_train)
y_test = enc.fit_transform(y_test)

print(x_train.shape)
print(x_test.shape)

x_train, y_train = np.concatenate((x_train, x_train), axis=0), np.concatenate((y_train, y_train), axis=0)
x_train, y_train = np.concatenate((x_train, x_train), axis=0), np.concatenate((y_train, y_train), axis=0)

print(x_train.shape)
print(x_test.shape)


## Inception v4 Implementation

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input, RandomFlip, RandomRotation, RandomZoom
from keras.metrics import TopKCategoricalAccuracy

resize_rescale = tf.keras.Sequential([
    Input(shape=(32, 32, 3)),
    Resizing(299, 299),
    Rescaling(1./255, offset=-1),
    RandomFlip('horizontal_and_vertical'),
    RandomRotation(0.2),
    RandomZoom(0.2)
])
# create the base pre-trained model
base_model = InceptionV3(input_tensor=resize_rescale.output, weights='imagenet', include_top=False)


# add own top to pre-trained model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

predictions = Dense(100, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# first train only the top layers
# freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=[
                  TopKCategoricalAccuracy(k=1, name='top_1_accuracy'),
                  TopKCategoricalAccuracy(k=5, name='top_5_accuracy'),
              ])




In [ ]:
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, class_names):
  """
  Returns a matplotlib figure containing the plotted confusion matrix.

  Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
  """
  figure = plt.figure(figsize=(8, 8))
  plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
  plt.title("Confusion matrix")
  plt.colorbar()
  tick_marks = np.arange(len(class_names))
  plt.xticks(tick_marks, class_names, rotation=45)
  plt.yticks(tick_marks, class_names)

  # Compute the labels from the normalized confusion matrix.
  labels = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

  # Use white text if squares are dark; otherwise black.
  threshold = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    color = "white" if cm[i, j] > threshold else "black"
    plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  return figure


def log_confusion_matrix(epoch, logs):
  # Use the model to predict the values from the validation dataset.
  test_pred_raw = model.predict(x_test)
  test_pred = np.argmax(test_pred_raw, axis=1)

  # Calculate the confusion matrix.
  cm = sklearn.metrics.confusion_matrix(y_test, test_pred)
  # Log the confusion matrix as an image summary.
  figure = plot_confusion_matrix(cm, class_names=np.arange(100))
  cm_image = plot_to_image(figure)

  # Log the confusion matrix as an image summary.
  with file_writer_cm.as_default():
    tf.summary.image("epoch_confusion_matrix", cm_image, step=epoch)

def plot_to_image(figure):
  """Converts the matplotlib plot specified by 'figure' to a PNG image and
  returns it. The supplied figure is closed and inaccessible after this call."""
  # Save the plot to a PNG in memory.
  buf = io.BytesIO()
  plt.savefig(buf, format='png')
  # Closing the figure prevents it from being displayed directly inside
  # the notebook.
  plt.close(figure)
  buf.seek(0)
  # Convert PNG buffer to TF image
  image = tf.image.decode_png(buf.getvalue(), channels=4)
  # Add the batch dimension
  image = tf.expand_dims(image, 0)
  return image

In [ ]:
from keras.models import load_model

model = load_model("/kaggle/working/logs/fit/20230529-180355/checkpoint/")

In [ ]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
# for i, layer in enumerate(base_model.layers):
#    print(i, layer.name)
from keras.metrics import TopKCategoricalAccuracy

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:255]:
   layer.trainable = False
for layer in model.layers[255:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=[
                  TopKCategoricalAccuracy(k=1, name='top_1_accuracy'),
                  TopKCategoricalAccuracy(k=5, name='top_5_accuracy'),
              ])
# run previous block to train model again, but with more epochs

In [ ]:
!pip install pyNVML
import pynvml as nvidia_smi

nvidia_smi.nvmlInit()

handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
total_gpu_memory = info.total
# Weights are float32 -> 4 bytes
model_size = model.count_params() * 4
# Rescaling layer returns float32
# 299*299 pixels, RGB colors
image_size = 4 * 299 * 299 * 3
MAX_BATCH_SIZE = int(np.floor(total_gpu_memory / (model_size + image_size)))
MAX_BATCH_SIZE_LOG2 = int(2 ** np.floor(np.log2(MAX_BATCH_SIZE)))
print(MAX_BATCH_SIZE)
print(MAX_BATCH_SIZE_LOG2)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, TerminateOnNaN
import datetime

logdir = "./logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

#cm_callback = keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.
model.fit(x_train, y_train,
          epochs=200,
          batch_size=MAX_BATCH_SIZE_LOG2,
          validation_batch_size=MAX_BATCH_SIZE_LOG2,
          validation_split=0.2,
          verbose=1,
          callbacks=[
              TensorBoard(
                  log_dir=logdir,
                  histogram_freq=1
              ),
              EarlyStopping(
                  monitor='val_top_1_accuracy',
                  patience=5,
              ),
              ModelCheckpoint(
                  logdir+'/checkpoint',
                  monitor='val_top_1_accuracy',
                  save_best_only=True,
              )
          ],
)


In [ ]:
for layer in model.layers[:199]:
   layer.trainable = False
for layer in model.layers[199:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=[
                  TopKCategoricalAccuracy(k=1, name='top_1_accuracy'),
                  TopKCategoricalAccuracy(k=5, name='top_5_accuracy'),
              ])

logdir = "./logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

#cm_callback = keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.
model.fit(x_train, y_train,
          epochs=200,
          batch_size=MAX_BATCH_SIZE_LOG2,
          validation_batch_size=MAX_BATCH_SIZE_LOG2,
          validation_split=0.2,
          verbose=1,
          callbacks=[
              TensorBoard(
                  log_dir=logdir,
                  histogram_freq=1
              ),
              EarlyStopping(
                  monitor='val_top_1_accuracy',
                  patience=5,
              ),
              ModelCheckpoint(
                  logdir+'/checkpoint',
                  monitor='val_top_1_accuracy',
                  save_best_only=True,
              )
          ],
)

In [ ]:
%load_external tensorboard

In [ ]:
%tensorboard --logdir ./logs/fit

In [ ]:
res = model.evaluate(x_test, y_test)

print(res)

In [ ]:
!pip install google
from google.colab import drive
drive.mount('/content/drive')